# Práctica 3: Grafos de conocimiento
#### (y algo de NLP)


**Ingeniería Electrónica**

**Inteligencia Artificial**

**28/04/2021**

El grafo de conocimiento es un concepto fascinante de la ciencia de datos. Es un método de representación del conocimiento mediante entidades interconectadas que pueden ser personas, ubicaciones, eventos, organizaciones, etc. 

Para construir un grafo de conocimiento, necesitamos formar **triples:sujeto, predicado y objeto** (o *entitad-propiedad-valor*) para vincular datos utilizando ontologías y semántica. 

En esta práctica, se utilizará una popular librería de Python para el procesamiento avanzado de lenguaje natural, spaCy.

## Librerías adicionales

⚠️ Ejecutar los siguientes comandos en la terminal para instalar las librerías necesarias:

   * conda install -c conda-forge spacy
   * python -m spacy download en_core_web_sm  (*procesamiento para idioma __inglés__*)
   * python -m spacy download es_core_news_sm  (*procesamiento para idioma __español__*)
   * pip install bs4 (puede estar ya incluido en anaconda)
   * pip install networkx (puede estar ya incluido en anaconda)

In [4]:
import spacy
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

Revisa la documentación de estas librerías para conocer sus funcionalidades.

## Visualizar entidades y relaciones

El procesamiento de texto con `nlp` devuelve un objeto `Doc` que contiene toda la información sobre los *tokens*, sus características lingüísticas y sus relaciones.

In [5]:
nlp = spacy.load('es_core_news_sm') # para procesamiento de textos en español

doc = nlp("El tipo lanzó el ladrillo")

for tok in doc:
  print(tok.text, "...", tok.dep_)

El ... det
tipo ... nsubj
lanzó ... ROOT
el ... det
ladrillo ... obj


Para visualizar en un cuaderno de Jupyter, se usa `displacy.render`.

In [6]:
displacy.render(doc, style="dep")

In [7]:
nlp = spacy.load('en_core_web_sm') # para procesamiento de textos en inglés

doc = nlp("If your hate could be turned into electricity, it would light up the whole world.")

for tok in doc:
  print(tok.text, "-->", tok.dep_)

If --> mark
your --> poss
hate --> nsubjpass
could --> aux
be --> auxpass
turned --> advcl
into --> prep
electricity --> pobj
, --> punct
it --> nsubj
would --> aux
light --> ROOT
up --> prt
the --> det
whole --> amod
world --> dobj
. --> punct


In [8]:
displacy.render(doc, style="dep")

In [9]:
nlp = spacy.load('es_core_news_sm')

doc = nlp("La imagen del agujero negro fue renderizada por la joven ingeniera.")

for tok in doc:
  print(tok.text, "-->", tok.dep_)

La --> det
imagen --> nsubj
del --> case
agujero --> nmod
negro --> amod
fue --> aux
renderizada --> ROOT
por --> case
la --> det
joven --> obj
ingeniera --> amod
. --> punct


In [10]:
displacy.render(doc, style="dep")

## Construir un grafo de conocimiento a partir de datos de texto

Lo construiremos en base un conjunto de datos, oraciones compiladas de artículos de wikipedia relacionados con la producción de cine. El archivo CSV de ejemplo está incluido en la carpeta de la práctica como "wiki_sentences.csv". Éstas oraciones están en inglés por lo que se debe cargar el modelo correspondente.

In [11]:
nlp = spacy.load('en_core_web_sm')

### Leer Datos desde CSV

In [12]:
# importar archivo con oraciones de wikipedia utilizando pandas (pd)
sentencias_candidatas = pd.read_csv("wiki_sentences.csv")
sentencias_candidatas.shape

(4318, 1)

In [13]:
# Realizar una muestra de 5 oraciones
sentencias_candidatas['sentence'].sample(5)

1089    in april 2017 fujifilm announced the instax sq...
3270    special make-up artist petr gorshenin, who wor...
3228    france, turman and schamus received final credit.
123     the  definitely wasn't there originally, and s...
3213              maggie awakens trapped in the basement.
Name: sentence, dtype: object

In [14]:
doc = nlp("today, the trend is for more shallow focus")

for tok in doc:
  print(tok.text, "-->", tok.dep_)

today --> npadvmod
, --> punct
the --> det
trend --> nsubj
is --> ROOT
for --> prep
more --> advmod
shallow --> amod
focus --> pobj


### Extracción de pares de entidades

Se pretende analizar una oración para extraer el sujeto y el objeto (las entidades) a medida que es leída. Sin embargo, existen algunos desafíos: una entidad puede abarcar varias palabras, y los analizadores de dependencia etiquetan solo las palabras individuales como sujetos u objetos.

La siguiente función extrae las entidades de una oración tomando en cuenta los problemas mencionados.

**Parte 1**
En la primera porción de código, se han definifo variables vacías. **prv_tok_dep** y **prv_tok_text** contendrán la etiqueta de dependencia de la palabra anterior en la oración y la palabra anterior en sí, respectivamente. **prefix** y **modifier** contendrán el texto asociado con el sujeto o el objeto.

**Parte 2**
A continuación, recorreremos los tokens en la oración. Primero verificaremos si el token es un signo de puntuación o no. En caso de serlo, lo ignoraremos y pasaremos al siguiente token. Si el token es parte de una palabra compuesta (etiqueta de dependencia = "*compound*"), lo mantendremos en la variable *prefix*. Una palabra compuesta es una combinación de varias palabras vinculadas para formar una palabra con un nuevo significado (ejemplo: "pista de patinaje", "amante de los libros").

**Parte 3**
Si el token es el sujeto, se capturará como la primera entidad en la variable **ent1**. Las variables prefix, modifier, prv_tok_dep y prv_tok_text se restablecen.

**Parte 4**
Si el token es el objeto, se capturará como la segunda entidad en la variable **ent2**. Las variables prefix, modifier, prv_tok_dep y prv_tok_text se restablecen.

**Parte 5**
Una vez que se haya capturado el sujeto y el objeto en la oración, actualizaremos el token anterior y su etiqueta de dependencia.

In [15]:
def get_entities(sent):
  ## Parte 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # etiqueta de dependencia del token anterior en la oración
  prv_tok_text = ""   # token anterior en la oración

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## Parte 2
    # Si el token es un signo de puntuación, pase al siguiente token
    if tok.dep_ != "punct":
      # comprobar: token es una palabra compuesta o no
      if tok.dep_ == "compound":
        prefix = tok.text
        # si la palabra anterior también era un 'compound', entonces agregar la palabra actual
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # comprobar: el token es un modificador o no
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # si la palabra anterior también era un 'compound', entonces agregar la palabra actual
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## Parte 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## Parte 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## Parte 5 
      # actualizar variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

Probamos esta función en una oración:

In [16]:
get_entities("An electric motor uses electrical energy")

['electric  motor', 'electrical  energy']

En la oración anterior, "*electric  motor*" es el sujeto y "*electrical energy*" es el objeto.

Ahora podemos usar esta función para extraer estos pares de entidades para todas las oraciones en nuestros datos:

In [17]:
pares_entidades = []

for i in tqdm(sentencias_candidatas["sentence"]): # con tqdm podemos mostrar una barra de progreso
  pares_entidades.append(get_entities(i))

100%|██████████| 4318/4318 [00:26<00:00, 164.52it/s]


La lista **pares_entidades** contiene todos los pares sujeto-objeto de las oraciones del archivo csv. Revisemos a algunos de ellos:

In [18]:
pares_entidades[10:20]

[['we', 'tests'],
 ['', 'international sales rights'],
 ['canadian musician robbie robertson', 'soundtrack'],
 ['it', 'original music tracks'],
 ['it', 'reviewed  franchise'],
 ['she', 'accidentally  mystique'],
 ['military  forces', 'arrest'],
 ['train', 'vuk'],
 ['kota eberhardt', 'telepath selene gallio'],
 ['singer', 'sequel']]

###  Extracción de relaciones / predicados

El predicado puede ser el verbo principal de una oración. La siguiente función utiliza la concordancia basada en reglas de spaCy para encontrar dichos predicados:

In [19]:
def get_relation(sent):

  doc = nlp(sent)

  # objeto de la clase Matcher
  matcher = Matcher(nlp.vocab)

  # definir el patrón 
  pattern = [[{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}]] 
    
  matcher.add("matching_1", pattern) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

El patrón definido en la función intenta encontrar la palabra raíz (ROOT) o el verbo principal en la oración. Una vez que se identifica el ROOT, el patrón verifica si es seguido por una preposición ("prep") o una palabra de agente. De ser así, se agrega a la palabra ROOT.

In [20]:
get_relation("An ECG detects the heartbeats.")

'detects'

Del mismo modo, se pueden obtener las relaciones de todas nuestras oraciones:

In [21]:
relaciones = [get_relation(i) for i in tqdm(sentencias_candidatas['sentence'])]

100%|██████████| 4318/4318 [00:26<00:00, 160.24it/s]


Revisamos a las relaciones o predicados más frecuentes que se acaban de extraer:

In [22]:
pd.Series(relaciones).value_counts()[:20]

is             348
was            283
released on     82
are             73
were            67
include         61
                50
's              41
released        39
have            31
has             29
became          29
released in     26
become          26
composed by     26
included        22
produced        21
called          21
been            20
considered      19
dtype: int64

Resulta que relaciones como "A es B" y "A era B" son las relaciones más comunes.

### Construir un grafo de conocimiento

Finalmente crearemos un grafoo de conocimiento a partir de las entidades extraídas (pares sujeto-objeto) y los predicados (relación entre entidades). Creamos un dataframe de entidades y predicados:

In [23]:
# extraer sujeto
fuente = [i[0] for i in pares_entidades]

# extraer objeto
objetivo = [i[1] for i in pares_entidades]

gc_df = pd.DataFrame({'fuente':fuente, 'objetivo':objetivo, 'arista':relaciones})

In [24]:
gc_df

,fuente,objetivo,arista
0,connie,own,decides
1,later scream,distance,heard in
2,christian,then elder,paralyzed by
3,temple,fire,set on
4,outside cult,him,wails with
...,...,...,...
4313,confidencial,negatively film,responded
4314,le parisien,five star rating,gave
4315,museum collection,"37,000 film titles",includes
4316,predecessor,historical film 1946,was


A continuación, utilizaremos la librería **networkx** para crear una red a partir de este dataframe. Los nodos representarán las entidades y las aristas o conexiones entre los nodos representarán las relaciones entre los nodos.

Será un grafo dirigido. En otras palabras, la relación entre cualquier par de nodos conectados no es bidireccional, es solo de un nodo a otro.

In [25]:
# crear un grafo dirigido desde un dataframe
G=nx.from_pandas_edgelist(gc_df, "fuente", "objetivo", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

Graficamos la red (esto puede tardar unos minutos):

In [ ]:
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

Resulta que hemos creado un gráfico con todas las relaciones que teníamos. Se vuelve realmente difícil visualizar un gráfico con muchas relaciones o predicados.

Por lo tanto, es recomendable utilizar solo unas pocas relaciones importantes para visualizar un gráfico, tomando una relación a la vez. Por ejemplo, con la relación "composed by":

In [ ]:
G=nx.from_pandas_edgelist(gc_df[gc_df['arista']=="composed by"], "fuente", "objetivo", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5) # k regula la distancia entre nodos
nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

Esa es una gráfica mucho más legible. Aquí las flechas apuntan hacia los compositores. Veamos algunas relaciones más. Qué se puede notar en los resultados?

In [ ]:
G=nx.from_pandas_edgelist(gc_df[gc_df['arista']=="written by"], "fuente", "objetivo", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5)
nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

In [ ]:
G=nx.from_pandas_edgelist(gc_df[gc_df['arista']=="released in"], "fuente", "objetivo", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

plt.figure(figsize=(12,12))
pos = nx.spring_layout(G, k = 0.5)
nx.draw(G, with_labels=True, node_color='skyblue', node_size=1500, edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

En este práctica, se revisó cómo extraer información de un texto dado en forma de triples (ontologías) y construir un gráfico de conocimiento a partir de él.

Sin embargo, nos limitamos a usar oraciones con exactamente 2 entidades. Incluso así pudimos construir gráficos de conocimiento bastante informativos. Consideren el potencial que tenemos aquí.

Se podría explorar más este campo de extracción de información para realizar la extracción de relaciones más complejas.